In [1]:
import pandas as pd
import numpy as np
import datetime
import dateutil.parser

In [9]:
def make_subdataset(elec, meta, weather_path,weather_file):
    one_year_hour = 365*24
#     weather = pd.read_csv(weather_path+weather_file,parse_dates=['timestamp'])
#     weather['rounded_timestamp'] = weather['timestamp'].apply(cutoff_minute)
#     weather = weather.groupby('rounded_timestamp').first() #only the first observation in each hour is taken
#     start_day_weather = weather.index[0]
#     weather = weather['TemperatureC']
    #construct the dataframe to return
    buildings = list(meta[(meta['newweatherfilename'] == weather_file)].index) #name of the buildings
    print(buildings)
    df = pd.DataFrame() #empty dataframe with 3 columns
    if len(buildings) > 0:
        for building in buildings:
            print(building)
            subdf = elec[['timestamp', building]]
            subdf.columns = ['timestamp', 'electricity']
            start_idx_building = elec[building].first_valid_index()
            last_idx_building = elec[building].last_valid_index()
            print(start_idx_building,last_idx_building,last_idx_building-start_idx_building)
            start_day = elec['timestamp'].loc[start_idx_building]
#             print(start_day_building, start_day_weather)
#             start_day = max(start_day_building, start_day_weather)
#             print(start_day)
            start_idx = elec[elec['timestamp']==start_day].index[0]
            subdf = subdf.iloc[start_idx:start_idx+one_year_hour]
            subdf['building_name'] = building
            subdf['weather_file'] = weather_file
            subdf['industry'] = meta.loc[building,'industry']
            subdf['area'] = meta.loc[building, 'sqm']
            subdf['primary_space_usage'] = meta.loc[building, 'primaryspaceuse_abbrev']
#             print(subdf['electricity'].iloc[-1])
#             print(subdf.shape)
            if np.isnan(subdf['electricity'].iloc[-1])==False:
                df = pd.concat([df, subdf], axis=0, ignore_index=True)
            else: 
                print('skipping')
    else:
        print('no buildings')
    return df

def add_onehot_dates(df):
    df = add_time_columns(df)
    df = pd.concat([df,pd.get_dummies(df['month'], prefix='month')],axis=1)
    df = pd.concat([df,pd.get_dummies(df['hour'], prefix='hour')],axis=1)
    df = pd.concat([df,pd.get_dummies(df['weekday'], prefix='weekday')],axis=1)
    return df

def cutoff_minute(dt):
    '''
    INPUT
    a datetime object has year, month, day, hour, and minute
    OUTPUT
    a datetime object has year, month, day, and hour
    '''
    year = dt.year
    month = dt.month
    day = dt.day
    hour = dt.hour
    return datetime.datetime(year, month, day, hour)

In [3]:
def one_hot_variable(df,features):
    for feat in features:
        df = pd.concat([df,pd.get_dummies(df[feat], prefix=feat)],axis=1)
    return df

In [4]:
def make_dataset(elec, meta, weather_path):
    import os
    meta = meta.set_index('uid')
    try:
        elec['timestamp'] = elec['timestamp'].apply(dateutil.parser.parse)
    except:
        pass
    elec['timestamp'] = elec['timestamp'].apply(cutoff_minute)
    weather_list = os.listdir(weather_path)
    df = pd.DataFrame()
    count = 1
    for weather_file in weather_list:
        print(weather_file + ', ' + str(count) + ' out of ' + str(len(weather_list)))
        count += 1
        subdf = make_subdataset(elec, meta, weather_path,weather_file)
        if type(df) != None:
            df = pd.concat([df, subdf], axis=0, ignore_index=True)
    df = add_onehot_dates(df)
    df = one_hot_variable(df, ['primary_space_usage','industry'])
    return df

In [5]:
def add_time_columns(df):
    year_list = [y.year for y in df['timestamp']]
    month_list = [y.month for y in df['timestamp']]
    hour_list = [y.hour for y in df['timestamp']]
    weekday_list = [y.weekday() for y in df['timestamp']]
    df['year'] = year_list
    df['month'] = month_list
    df['hour'] = hour_list
    df['weekday'] = weekday_list
    return df.sort_values(['building_name', 'timestamp'])

In [6]:
meta = pd.read_csv('../the-building-data-genome-project-master/data/raw/meta_open.csv')
elec = pd.read_csv('../the-building-data-genome-project-master/data/processed/temp_open_utc_complete.csv',parse_dates=['timestamp'])

In [7]:
weather_path = '../the-building-data-genome-project-master/data/external/weather/'

In [10]:
df_test = make_dataset(elec, meta, weather_path)

weather0.csv, 1 out of 39
['UnivClass_Amari', 'UnivClass_Ariana', 'Office_Ayesha', 'UnivDorm_Adan', 'Office_Allyson', 'Office_Amelie', 'UnivClass_Alexander', 'Office_Ava', 'PrimClass_Angela', 'Office_Abbey', 'UnivDorm_Ahmad', 'UnivClass_Armando', 'UnivDorm_April', 'UnivClass_Anamaria', 'UnivClass_Andy', 'UnivClass_Archie', 'UnivDorm_Ashleigh', 'UnivClass_Alec', 'UnivClass_Alvin', 'Office_Alyson', 'UnivDorm_Alonzo', 'UnivClass_Abraham', 'Office_Aubrey', 'Office_Autumn', 'Office_Abigail', 'UnivClass_Alexandra', 'UnivDorm_Antonio', 'UnivClass_Allen', 'UnivDorm_Alyssa', 'UnivClass_Axel', 'Office_Annika', 'UnivDorm_Avery', 'UnivClass_Annmarie', 'Office_Asher', 'Office_Al', 'Office_Andrea', 'UnivClass_Ayanna', 'UnivClass_Amya', 'UnivDorm_Alka', 'UnivLab_Alexis', 'UnivClass_Alexandria', 'UnivClass_Annabella', 'UnivClass_Antoinette', 'Office_Angelo', 'Office_Anastasia', 'UnivClass_Aoibhe', 'UnivClass_Anya', 'Office_Amelia', 'UnivClass_Alicia', 'Office_Angelina', 'UnivClass_Alexus', 'UnivClass_

31429 40188 8759
PrimClass_Jensen
31429 40188 8759
PrimClass_Jeffrey
31429 40188 8759
PrimClass_Jennifer
31429 40188 8759
PrimClass_Jesse
31429 40188 8759
PrimClass_Jaxson
31429 40188 8759
PrimClass_Jimmie
31429 40188 8759
Office_Jayden
31429 40188 8759
PrimClass_Jayda
31429 40188 8759
PrimClass_Jocelyn
31429 40188 8759
PrimClass_Jeffery
31429 40188 8759
PrimClass_Jayson
31429 40188 8759
UnivDorm_Jeannette
31429 40188 8759
PrimClass_Jennie
31429 40188 8759
PrimClass_Jonathon
31429 40188 8759
PrimClass_Jaqueline
31429 40188 8759
PrimClass_Jeremy
31429 40188 8759
PrimClass_Josue
31429 40188 8759
PrimClass_Julio
31429 40188 8759
PrimClass_Jacquelyn
31429 40188 8759
PrimClass_Juanita
31429 40188 8759
PrimClass_Joel
31429 40188 8759
PrimClass_Joey
31429 40188 8759
PrimClass_Joanna
31429 40188 8759
PrimClass_Jediah
31429 40188 8759
PrimClass_Janelle
31429 40188 8759
PrimClass_Jane
31429 40188 8759
PrimClass_Jon
31429 40188 8759
PrimClass_Jake
31429 40188 8759
Office_Joan
31429 40188 8759
Pri

weather27.csv, 21 out of 39
['Office_Eileen']
Office_Eileen
8763 17546 8783
weather28.csv, 22 out of 39
['Office_Ellie']
Office_Ellie
8763 17546 8783
weather29.csv, 23 out of 39
['PrimClass_Ezekiel']
PrimClass_Ezekiel
8760 17543 8783
weather3.csv, 24 out of 39
['Office_Marla', 'Office_Maryann', 'Office_Myron', 'Office_Mick', 'Office_Moises', 'UnivDorm_Marlene', 'Office_Marcus', 'Office_Marc', 'Office_Mary', 'Office_Monty', 'Office_Marlon', 'Office_Maximus', 'Office_Michael', 'Office_Moses', 'Office_Martha', 'Office_Mercedes', 'Office_Mat', 'Office_Marilyn', 'UnivDorm_Malachi', 'UnivDorm_Mathew', 'Office_Malik', 'UnivLab_Miles', 'Office_Mohammed', 'UnivDorm_Mitch', 'Office_Micheal', 'Office_Mark', 'UnivDorm_Malcolm', 'Office_Max', 'UnivDorm_Marquis', 'UnivDorm_Mauricio', 'UnivClass_Maddison', 'Office_Mada', 'UnivDorm_Mckenzie', 'Office_Marianne', 'Office_Marvin', 'Office_Maya', 'Office_Matthew', 'Office_Madeleine', 'Office_Melinda', 'Office_Megan', 'Office_Madisyn', 'Office_Morgan', 'Of

UnivClass_Stephanie
31429 40188 8759
UnivLab_Suzette
31429 40188 8759
UnivLab_Santiago
31429 40188 8759
Office_Shelly
31429 40188 8759
weather7.csv, 37 out of 39
['UnivLab_Bethany', 'UnivClass_Boyd', 'Office_Bobbi', 'Office_Benjamin', 'UnivClass_Beatrice', 'Office_Bianca', 'Office_Benthe', 'Office_Brian', 'UnivLab_Beth', 'UnivClass_Bob', 'UnivLab_Brad', 'UnivClass_Brett', 'UnivLab_Bert', 'Office_Bryon', 'UnivLab_Brenna']
UnivLab_Bethany
8763 17546 8783
UnivClass_Boyd
8763 17546 8783
Office_Bobbi
8763 17546 8783
Office_Benjamin
8763 17546 8783
UnivClass_Beatrice
8763 17546 8783
Office_Bianca
8763 17546 8783
Office_Benthe
8763 17546 8783
Office_Brian
8763 17546 8783
UnivLab_Beth
8763 17546 8783
UnivClass_Bob
8763 17546 8783
UnivLab_Brad
8763 17546 8783
UnivClass_Brett
8763 17546 8783
UnivLab_Bert
8763 17546 8783
Office_Bryon
8763 17546 8783
UnivLab_Brenna
8763 17546 8783
weather8.csv, 38 out of 39
['UnivClass_Tamra', 'UnivClass_Tammy', 'UnivClass_Therese', 'Office_Tod', 'UnivClass_Teri',

In [11]:
lengths_test = []
for bldg in df_test['building_name'].unique():
    lengths_test.append([bldg,len(df_test[df_test['building_name']==bldg])])

In [12]:
count = 0
for l in lengths_test:
    if l[1] > 8759:
        count += 1

In [14]:
df_test.to_csv('data_full_noTemp.csv')

In [13]:
count

507

In [63]:
df_noweather3 = make_dataset(elec, meta, weather_path)

weather0.csv, 1 out of 39
UnivClass_Amari
32180 40939 8759
weather1.csv, 2 out of 39
PrimClass_Jolie
31429 40188 8759
weather10.csv, 3 out of 39
PrimClass_Uma
9515 18274 8759
weather11.csv, 4 out of 39
PrimClass_Evie
8761 17544 8783
weather12.csv, 5 out of 39
PrimClass_Everett
8760 17543 8783
weather13.csv, 6 out of 39
Office_Dawn
0 8759 8759
weather14.csv, 7 out of 39
PrimClass_Eoghan
8760 17543 8783
weather15.csv, 8 out of 39
PrimClass_Elliott
8760 17543 8783
weather16.csv, 9 out of 39
PrimClass_Ervin
8760 17543 8783
weather17.csv, 10 out of 39
PrimClass_Esmeralda
8760 17543 8783
weather18.csv, 11 out of 39
PrimClass_Ethel
8762 17545 8783
weather19.csv, 12 out of 39
PrimClass_Ellen
8761 17544 8783
weather2.csv, 13 out of 39
UnivClass_Clifford
32178 40937 8759
weather20.csv, 14 out of 39
Office_Evelyn
8763 17546 8783
weather21.csv, 15 out of 39
PrimClass_Ernest
8761 17544 8783
weather22.csv, 16 out of 39
Office_Elizabeth
8763 17546 8783
weather23.csv, 17 out of 39
PrimClass_Elisabeth


In [26]:
df.head()

,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,TemperatureC,year,month,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
9,2015-01-01 07:00:00,27.79,Office_Abbey,weather0.csv,Education,6891.54454,Office,2.8,2015,1,...,0,0,1,0,0,0,0,0,1,0
105,2015-01-01 08:00:00,27.66,Office_Abbey,weather0.csv,Education,6891.54454,Office,2.8,2015,1,...,0,0,1,0,0,0,0,0,1,0
201,2015-01-01 09:00:00,27.13,Office_Abbey,weather0.csv,Education,6891.54454,Office,3.9,2015,1,...,0,0,1,0,0,0,0,0,1,0
297,2015-01-01 10:00:00,27.34,Office_Abbey,weather0.csv,Education,6891.54454,Office,5.0,2015,1,...,0,0,1,0,0,0,0,0,1,0
393,2015-01-01 11:00:00,27.34,Office_Abbey,weather0.csv,Education,6891.54454,Office,5.6,2015,1,...,0,0,1,0,0,0,0,0,1,0


In [12]:
df.to_csv('full_dataset.csv')

In [13]:
df2 = df.set_index(['building_name','timestamp']).unstack(1)

In [16]:
df2.to_csv('full_dataset_orderedbyrow.csv')

In [17]:
df3 = df.set_index(['building_name','timestamp'])

In [18]:
df3.to_csv('full_dataset_indexbldgtime.csv')